In [1]:
!pip install langchain langchain-community langchain-core faiss-cpu pypdf pyngrok PyPDF2 sentence-transformers transformers==4.52.4 

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 103.0 MB/s eta 0:00:0000:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.9/457.9 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 92.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00

## Loading & Importings

In [2]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
import torch
import re
from langchain.document_loaders import PyPDFLoader

## Loading Mistral

In [3]:
model_name = "mistralai/Mistral-Nemo-Instruct-2407"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

2026-01-11 14:19:20.925591: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768141161.162456      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768141161.229710      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


...

model-00002-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [4]:
def generate_text(prompt, max_length=1000, num_return_sequences=1):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

## Schema

In [5]:
name_schema = ResponseSchema(
    name = "full_name",
    description = "Full name of the Candidate as mentioned in the documnet"
)

email_schema = ResponseSchema(
    name = "email",
    description = "The Candidate's email in the document"
)

education_schema = ResponseSchema(
    name = "Education",
    description = "list of information about candidate education, the earned degree, the institution, and graduation year"
)

skills_schema = ResponseSchema(
    name = "Skills",
    description = "list of soft and technical skills mentioned in the candidate CV"
)

experience_schema = ResponseSchema(
    name = "Experience",
    description = "list of candidate's experiences: what is his role, where does he work, and for how many years"
)

response_schema = [name_schema,email_schema,education_schema,skills_schema,experience_schema]

In [6]:
output_parser = StructuredOutputParser.from_response_schemas(response_schema)
format_instructions = output_parser.get_format_instructions()

In [7]:
purchase_extraction_template = """
You are a smart assistant that extracts Candidate information from the CV sent to the HR.

Extract the candidate name, email, education, skills and previous experience.

Respond ONLY in JSON format as follows:
{format_instructions}

Example Input:
'''
John Smith – john.smith@email.com
Education: B.Sc. Computer Science, MIT, 2020
Skills: Python, Machine Learning, Data Analysis
Experience:
- Software Engineer at Google (2020–2023)
- Data Scientist at OpenAI (2023–Present)
'''

Now extract from the following input:
"{user_input}"
"""

In [8]:
def extract_json_block(text):
    pattern = r'```json\s*(.*?)\s*```'
    matches = re.findall(pattern, text, re.DOTALL)
    return matches[-1].strip() if matches else text.strip()

## Deployment

In [9]:
NGROK_TOKEN = "386REcz4JsU2OpwGtAeOqsd8H7j_81puHo7ELtjbx6QxVhpZp"
API_KEY = "secret123"

In [10]:
from PyPDF2 import PdfReader
import io

def extract_text_from_bytes(pdf_bytes):
    pdf_file = io.BytesIO(pdf_bytes)
    reader = PdfReader(pdf_file)
    text = ""
    for page in reader.pages:
        page_text = page.extract_text() or ""
        text += page_text + "\n"
    return text

In [11]:
from fastapi import FastAPI, Request, HTTPException
import uvicorn, threading, time, socket
from pyngrok import ngrok, conf
import base64
import json


app = FastAPI()

@app.post("/CV_Extractor")
async def rag(req: Request):
    if req.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401, detail="Unauthorized")
    data = await req.json()
    pdf = data.get("pdf_file")
    pdf_bytes = base64.b64decode(pdf)
    pdf_text = extract_text_from_bytes(pdf_bytes)
    prompt = PromptTemplate(
        template=purchase_extraction_template,
        input_variables=["user_input", "format_instructions"]).format(user_input=pdf_text, format_instructions=format_instructions)
    res = generate_text(prompt, max_length=3000)[0]  # النتيجة raw string
    json_text = extract_json_block(res)
    output_data = output_parser.parse(json_text)
    return {
        "response": output_data
    }

In [13]:
import socket
import asyncio
import uvicorn
from pyngrok import ngrok, conf

def free_port():
    s = socket.socket()
    s.bind(('', 0))
    port = s.getsockname()[1]
    s.close()
    return port

port = free_port()

# ngrok
conf.get_default().auth_token = NGROK_TOKEN
public_url = ngrok.connect(port).public_url
print("Your public URL:", public_url)

# uvicorn داخل نفس event loop بتاع النوتبوك
config = uvicorn.Config(app, host="0.0.0.0", port=port, log_level="info")
server = uvicorn.Server(config)
asyncio.get_running_loop().create_task(server.serve())

Your public URL: https://pennie-sabulous-rheba.ngrok-free.dev


<Task pending name='Task-4' coro=<Server.serve() running at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:69>>

INFO:     Started server process [47]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:46703 (Press CTRL+C to quit)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


INFO:     154.236.156.93:0 - "POST /CV_Extractor HTTP/1.1" 200 OK


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


INFO:     154.236.156.93:0 - "POST /CV_Extractor HTTP/1.1" 200 OK
